#### Reading instance and defining model

In [2468]:
# Reading instance

from instanceReaderSMDHPDPTW import Instance

instanceName = "AA10 - Modificada"

MV = 12

n_pickup_sites = 2

fleet_capacities = [10, 20]

vehicle_locations= [[0], [1]]

inst = Instance(instanceName,
                fleet_capacities,
                n_pickup_sites,
                multiple_depots=True,
                vehicle_locations=[[0], [1]])

    solver failure.


In [2469]:
# Applying modifications - These would be more suitable on reader!
# but for now, I'll do it like that

# Instance data
df_instance = inst.instance_data
df_instance = df_instance[['type','x','y','d','w_a','w_b']]

# Taking away artificial nodes
valid_types = ['Depot', 'Pickup','Delivery']
df_instance = df_instance[df_instance['type'].isin(valid_types)].reset_index(drop=True)

# Dropping duplicate pickup sites
df_instance = df_instance.drop_duplicates(['type','x']).reset_index(drop=True)

# Multiplying demands by factor
pickup_factor = 20
delivery_factor = 5
df_instance['d'] = [demand*pickup_factor if (demand > 0) else demand*delivery_factor for demand in df_instance['d']]

# Appending back depot nodes at the end
df_instance = df_instance.append(df_instance[df_instance['type'] == 'Depot']).reset_index(drop=True)

In [2470]:
# Distances
c = [[0 for i in range(len(df_instance))] for j in range(len(df_instance))]

for i in range(len(df_instance)):
    for j in range(len(df_instance)):
        
        x_i = df_instance.iloc[i]['x']
        x_j = df_instance.iloc[j]['x']
        y_i = df_instance.iloc[i]['y']
        y_j = df_instance.iloc[j]['y']
        
        dist = round(((x_i-x_j)**2 + (y_i-y_j)**2)**(1/2),2)
        c[i][j] = dist
        
# Demands
d = df_instance['d']

# Time windows
w_a = df_instance['w_a']
w_b = df_instance['w_b']

# Capacities
Q = fleet_capacities

In [2471]:
import pandas as pd

# Importing pyomo modules

import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Creating model
model = pyo.ConcreteModel()

In [2472]:
# Visualizing instance

df_instance.set_index(pd.Index(range(1, len(df_instance) + 1)))

,type,x,y,d,w_a,w_b
1,Depot,19.942,16.269,0.0,0.0,1000.0
2,Depot,8.195,24.765,0.0,0.0,1000.0
3,Pickup,19.942,16.269,140.0,0.0,1000.0
4,Pickup,8.195,24.765,100.0,0.0,1000.0
5,Delivery,35.573,46.733,-35.0,209.0,269.0
6,Delivery,36.158,11.897,-45.0,562.0,622.0
7,Delivery,43.760,46.624,-75.0,261.0,321.0
8,Delivery,13.387,36.048,-25.0,549.0,609.0
9,Delivery,21.442,34.131,-60.0,470.0,530.0
10,Delivery,4.515,24.464,-25.0,439.0,499.0


In [2473]:
tw_factor = 0.4

w_b = [tw*tw_factor if tw == 1000 else tw for tw in w_b]

w_b

[400.0,
 400.0,
 400.0,
 400.0,
 269.0,
 622.0,
 321.0,
 609.0,
 530.0,
 499.0,
 605.0,
 494.0,
 117.0,
 185.0,
 400.0,
 400.0]

## Defining sets and notation

## $  N = S_{0} \cup P \cup D \cup S_{f} = \{1, ..., 3s + n\} $
## $ S_{0} = \{1, ... , s\} $
## $ P = \{s + 1, ... , 2s\} $
## $ D = \{2s + 1, ... , 2s + n\} $
## $ S_{f} = \{2s + n + 1, ... , 3s + n\} $

#### Vehicles set

## $ \newline K = \{1, ..., m\} $

#### Visits set

## $ \newline V = \{1, ..., MV\} $


In [2474]:
# Sets

model.setN = pyo.Set(initialize=range(len(df_instance)))
model.setP = pyo.Set(initialize=df_instance[df_instance['type'] == 'Pickup'].index)
model.setD = pyo.Set(initialize=df_instance[df_instance['type'] == 'Delivery'].index)

# Additional sets

number_of_depots = int(len(df_instance[df_instance['type'] == 'Depot'])/2)
model.setS_0 = pyo.Set(initialize=df_instance[df_instance['type'] == 'Depot'].iloc[:number_of_depots].index)
model.setS_f = pyo.Set(initialize=df_instance[df_instance['type'] == 'Depot'].iloc[-number_of_depots:].index)
model.setS = (model.setS_0 | model.setS_f)


# Set of vehicles
model.setK = pyo.Set(initialize=range(len(fleet_capacities)))

# Set of visits
model.setV = pyo.Set(initialize=range(MV))

#### Defining variables

### $x_{ijkv} = \text{1 if vehicle $k$ travels from $i$ to $j$ in its $v$ visit and 0 otherwise;} $

### $y_{ikv} = \text{amount of load in vehicle $k$ after visiting node $i$ in its $v$ visit;} $

### $w_{ikv} = \text{arrival time of vehicle $k$ in node $i$ in its $v$ visit;} $

### $z_{ikv} = \text{amount of load supplied/delivered by vehicle $k$ to node $i$ in its $v$ visit;} $

In [2475]:
# Variables

model.x = pyo.Var(model.setN, model.setN, model.setK, model.setV, within = pyo.Binary)

model.y = pyo.Var(model.setN, model.setK, model.setV, within = NonNegativeReals)

model.w = pyo.Var(model.setN, model.setK, model.setV, within = NonNegativeReals)

model.z = pyo.Var(model.setN, model.setK, model.setV, within = NonNegativeReals)

#### Defining constraints

#### Objective function

## $ (1) $

## $$ max \displaystyle \sum_{i \in D}\displaystyle \sum_{i \in K}\displaystyle \sum_{j \in V}z_{ikv}$$

In [2476]:
# (1) - Objective function

model.exprobj = sum(sum(sum(model.z[i,k,v] for i in model.setD) for k in model.setK) for v in model.setV)

model.obj = pyo.Objective(expr= model.exprobj, sense = pyo.maximize)

## $ (2) $

## $$\displaystyle \sum_{i \in N}\displaystyle \sum_{v \in V}x_{ijkv} \leq \frac{MV}{2}, \hspace{15pt} \forall j \in P \cup D, k \in K$$

In [2477]:
# (2) - Maximum number of visits in a node must be MV/2, which would be sucessive back and forth trips

model.R2 = pyo.ConstraintList()

for j in (model.setP | model.setD):
    
    for k in model.setK:
    
        model.R2.add(expr = sum(sum(model.x[i,j,k,v] for i in model.setN) for v in model.setV) <= MV/2)

## $ (3) $


## $$ \displaystyle \sum_{i \in N \setminus S_{f}}x_{ihk(v-1)} -  \displaystyle \sum_{j \in N \setminus S_{0}}x_{hjkv} = 0, \hspace{15pt} \forall h \in P \cup D, k \in K, v \in V \setminus \{0\} $$

In [2478]:
# (3) - Flow conservation - if vehicle k goes from i to h, it has to go from h to j on the next visit

model.R3 = pyo.ConstraintList()

for h in (model.setP | model.setD):
    
    for k in model.setK:
        
        for v in range(1,len(model.setV)):

            model.R3.add(expr = sum(model.x[i,h,k,v-1] for i in (model.setN - model.setS_f)) - sum(model.x[h,j,k,v] for j in (model.setN - model.setS_0)) == 0)


## $ (4) $



## $$ \displaystyle \sum_{i \in N}\sum_{j \in N}\sum_{v \in V}x_{ijkv} \leq MV, \hspace{15pt} \forall k \in K$$

In [2479]:
# (4) - Vehicles can visit "MV" arcs in their routes at most

model.R4 = pyo.ConstraintList()
    
for k in model.setK:

        model.R4.add(expr = sum(sum(sum(model.x[i,j,k,v] for i in model.setN) for j in model.setN) for v in model.setV) <= MV)

## $ (5) $



## $$ \displaystyle x_{ijk0} = 0, \hspace{15pt} \forall i \in N \setminus S_{0} , j \in N , k \in K $$

In [2480]:
# (5) - Vehicles must depart from depot

model.R5 = pyo.ConstraintList()

for i in (model.setN - model.setS_0):
    
    for j in model.setN:
    
        for k in model.setK:
        
            model.R5.add(expr = model.x[i,j,k,0] == 0)

## $ (6) $



## $$ \displaystyle x_{ijkMV} = 0, \hspace{15pt} \forall j \in N \setminus S_{f} , i \in N , k \in K $$

In [2481]:
# (6) - Vehicles must end their routes in one of the available depots

model.R6 = pyo.ConstraintList()

for i in model.setN:
    
    for j in (model.setN - model.setS_f):
    
        for k in model.setK:
        
            model.R6.add(expr = model.x[i,j,k,model.setV.bounds()[1]] == 0)

## $ (7) $


## $$ \displaystyle x_{ijk0} = 0,  \hspace{15pt} \forall i \in S_{0}; j \in N \setminus D_{i}; k \in K$$

In [2482]:
# (7) - A vehicle can't depart from a depot that is not its initial depot

model.R7 = pyo.ConstraintList()

for i in model.setS_0:

    for j in model.setN:
        
        for k in model.setK:
            
            if (k not in vehicle_locations[i]):
                       
                model.R7.add(expr = model.x[i, j, k, 0] == 0) 

## $ (8) $

## $$ \displaystyle \sum_{j \in N}\sum_{k \in K}x_{ijk0} \leq K_{i}, \hspace{15pt} \forall i \in S_{0} $$

In [2483]:
# (8) - Number of vehicles that depart from depot must be equal to number of vehicles available at depot

# K_{i} - Number of vehicles that start their route at depot i in S_0

K = [len(location) for location in vehicle_locations]

model.R8 = pyo.ConstraintList()

for i in (model.setS_0):

    model.R8.add(expr = sum(sum(model.x[i,j,k,0] for j in model.setN) for k in model.setK) <= K[i])

## $ (9) $



## $$ \displaystyle x_{ijkv} = 0, \hspace{15pt} \forall i \in S_{0}, j \in D , k \in K, v \in V $$

In [2484]:
# (9) - Vehicles can't go from depot directly to a delivery node (was this supposed to be a constraint?)

# Y and Z variables are kind of problematic for initial visit, so this is needed

model.R9 = pyo.ConstraintList()

for i in model.setS_0:
    
    for j in model.setD:
    
        for k in model.setK:
            
            for v in model.setV:
        
                model.R9.add(expr = model.x[i,j,k,v] == 0)

## $ (10) $

## $$ w_{jkv} \geq w_{ik(v-1)} + t_{ij} - M(1-x_{ijkv}), \newline\forall i \in N, j \in N, k \in K, v \in V \setminus\{0\} $$

In [2485]:
# (10) - Time windows constraints - 1

# Defining big M

M = 1000

model.R10 = pyo.ConstraintList()

for i in model.setN:
    
    for j in model.setN:
        
        for k in model.setK:
    
            for v in range(1, len(model.setV)):
        
                model.R10.add(expr = model.w[j, k, v] >= model.w[i, k, v-1] + c[i][j] - M*(1 - model.x[i,j, k, v]))
            
                model.R10.add(expr = model.w[j, k, v] <= model.w[i, k, v-1] + c[i][j] + M*(1 - model.x[i,j, k, v]))

## $ (11) $



## $$ w_{i}^{a} \leq w_{ikv} \leq w_{i}^{b} , \hspace{15pt} \forall i \in P \cup D, k \in K, v \in V $$

In [2486]:
# (11)  - Time windows constraints - 2

model.R11 = pyo.ConstraintList()

for i in (model.setP | model.setD):
    
    for k in model.setK:
        
        for v in model.setV:
    
            model.R11.add(expr = model.w[i, k, v] >= w_a[i])
            model.R11.add(expr = model.w[i, k, v] <= w_b[i])

## $ (12) $



## $$ y_{jkv} \geq y_{ik(v-1)} + z_{jkv} - M(1-x_{ijkv}), \newline\forall i \in N, j \in P, k \in K, v \in V \setminus \{0\} $$

In [2487]:
# (12) - Load constraints - 1 (pickup)

model.R12 = pyo.ConstraintList()

for i in model.setN:
    
    for j in model.setP:
        
        for k in model.setK:
    
            for v in range(1, len(model.setV)):
        
                model.R12.add(expr = model.y[j,k,v] >= model.y[i,k,v-1] + model.z[j,k,v] - M*(1 - model.x[i,j,k,v]))
            
                model.R12.add(expr = model.y[j,k,v] <= model.y[i,k,v-1] + model.z[j,k,v] + M*(1 - model.x[i,j,k,v]))
                
# (12) - Load constraints - 1 (pickup)

for i in model.setN:
        
    for k in model.setK:

            model.R12.add(expr = model.y[i,k,0] >= model.z[i,k,0])

            model.R12.add(expr = model.y[i,k,0] <= model.z[i,k,0])

## $ (13) $



## $$ y_{jkv} \geq y_{ik(v-1)} - z_{jkv} - M(1-x_{ijkv}), \newline\forall i \in N, j \in D, k \in K, v \in V \setminus \{0\} $$

In [2488]:
# (13)  - Load constraints - 2 (delivery)

model.R13 = pyo.ConstraintList()

for i in model.setN:
    
    for j in model.setD:
        
        for k in model.setK:
    
            for v in range(1, len(model.setV)):
            
                model.R13.add(expr = model.y[j,k,v] >= model.y[i,k,v-1] - model.z[j,k,v] - M*(1 - model.x[i,j,k,v]))
                
                model.R13.add(expr = model.y[j,k,v] <= model.y[i,k,v-1] - model.z[j,k,v] + M*(1 - model.x[i,j,k,v]))

## $ (14) $



## $$ \displaystyle y_{ikv} \leq Q_{k}, \hspace{15pt} \forall i \in N , k \in K, v \in V$$

In [2489]:
# (14) - Load constraints - 3

model.R14 = pyo.ConstraintList()

for i in model.setN:

        for k in model.setK:

            for v in model.setV:

                model.R14.add(expr = model.y[i,k,v] <= Q[k])


## $ (15) $


## $$ \displaystyle \sum_{k \in K}\sum_{v \in V} z_{ikv} \leq d_{i}, \hspace{15pt} \forall i \in P \cup D $$

In [2490]:
# (15)  - Total amount picked up/delivered to each node can't be above node supply/demand

model.R15 = pyo.ConstraintList()

for i in (model.setP | model.setD):

    model.R15.add(expr = sum(sum(model.z[i, k, v] for k in model.setK) for v in model.setV) <= abs(d[i]))

## $ (16) $


## $$  z_{ikv} \leq d_{i}\displaystyle \sum_{j \in N}x_{jikv}, \hspace{15pt} \forall i \in N, k \in K, v \in V $$

In [2491]:
# (16) - Amount delivered to a node in a single visit can't exceed node demand

model.R16 = pyo.ConstraintList()

for i in model.setN:
    
    for k in model.setK:
        
        for v in model.setV:

            model.R16.add(expr = model.z[i, k, v] <= abs(d[i])*sum(model.x[j,i,k,v] for j in model.setN))

## $ (17) $


## $$  \displaystyle \sum_{i \in P}\sum_{v \in V} z_{ikv} =\displaystyle \sum_{i \in D}\sum_{v \in V}z_{ikv}, \forall k \in K$$

In [2492]:
# (17) - Total delivered amount by a vehicle needs to be equal to total picked up demand

model.R17 = pyo.ConstraintList()

for k in model.setK:
    
    model.R17.add(expr = sum(sum(model.z[i, k, v] for i in model.setP) for v in model.setV) == sum(sum(model.z[i, k, v] for i in model.setD) for v in model.setV))


## $ (18) $


## $$  z_{ik0} \leq Q_{k}, \hspace{15pt} \forall i \in N, k \in K$$

In [2493]:
# # (18)  - Amount collected/delivered at the first visit can't exceed vehicle capacity

# # Also needed because of problems with Y and Z for first visit

# model.R18 = pyo.ConstraintList()

# for i in model.setN:
    
#     for k in model.setK:

#         model.R18.add(expr = model.z[i,k,0] <= Q[k])

In [2494]:
# # (19) - Forces that a vehicle can't go to a pickup node that is not in the same place of its initial depot

# # Can be turned off, if convenient in some way

# model.R22 = pyo.ConstraintList()

# for i in model.setS_0:

#     for j in model.setP:
        
#         for k in model.setK:
            
#             if (c[i][j] > 0):

#                 model.R22.add(expr = model.x[i, j, k, 0] == 0)

#### Valid inequalities

In [2495]:
# # Inequalities for N set

model.VI1 = pyo.ConstraintList()

# No "node to initial depot" arcs or "final depot to nodes" arcs

for i in model.setN:
    
    for k in model.setK:
        
        for v in model.setV:
    
            #model.VI1.add(expr= model.x[i, i, k, v] == 0)

            for h in model.setS_0:

                model.VI1.add(expr= model.x[i, h, k, v] == 0)

            for h in model.setS_f:

                model.VI1.add(expr= model.x[h, i, k, v] == 0)
                

# Vehicles can't visit pickup node right before ending route

model.RV2 = pyo.ConstraintList()

for i in model.setP:
    
    for j in model.setS_f:
    
        for k in model.setK:
        
            model.RV2.add(expr = model.x[i,j,k,model.setV.bounds()[1]] == 0)


# Time windows inequalities

model.VI3 = pyo.ConstraintList()

for i in (model.setP | model.setD):
    
    for j in (model.setP | model.setD):
        
        for k in model.setK:
            
            for v in model.setV:
        
                if w_a[i] + c[i][j] > w_b[j]:

                    model.VI3.add(expr= model.x[i, j, k, v] == 0)

                    
# Delivery capacities inequalities

model.VI4 = pyo.ConstraintList()

for i in model.setD:
    
    for j in model.setD:
        
        for k in model.setK:
            
            for v in model.setV:
        
                if (abs(d[i]) + abs(d[j]) > Q[k]):
            
                    model.VI4.add(expr= model.x[i, j, k, v] == 0)
                
# # (ZZ) - No "initial depot to final depot" arcs

# model.RZZ = pyo.ConstraintList()

# for i in model.setS_0:
    
#     for j in model.setS_f:
        
#         for k in model.setK:

#             for v in model.setV:

#                 model.RZZ.add(expr = model.x[i, j, k, v] == 0)
        

In [2496]:
# (YY)

model.RYY = pyo.ConstraintList()
        
for k in model.setK:
    
    for v in model.setV:

        model.RYY.add(expr = sum(sum(model.x[i, j, k, v] for i in model.setN) for j in model.setN) <= 1)

In [2498]:
model.RMM = pyo.ConstraintList()

for i in model.setN:
        
    for k in model.setK:
        
        for v in model.setV:

            model.RMM.add(expr = model.z[i,k,v] <= M*(1-model.x[i, i, k, v]))
            model.RMM.add(expr = model.z[i,k,v] >= (-1)*M*(1-model.x[i, i, k, v]))
        

In [2499]:
# No nodes going to final depot in visits lower than MV

model.RXYZ = pyo.ConstraintList()

for i in model.setN:
    
    for j in model.setS_f:
        
        for k in model.setK:

            for v in range(len(model.setV) - 1):

                model.RXYZ.add(expr = model.x[i, j, k, v] == 0)
                
# No nodes starting on initial depots in visits higher than 0

for i in model.setS_0:
    
    for j in model.setN:
        
        for k in model.setK:

            for v in range(1, len(model.setV)):

                model.RXYZ.add(expr = model.x[i, j, k, v] == 0)


In [ ]:
# "Equity" constrain

#### Solving model

In [2501]:
# Solver

TimeLimit = 600

opt = pyo.SolverFactory('cplex', executable='C:/Program Files/IBM/ILOG/CPLEX_Studio201/cplex/bin/x64_win64/cplex')
opt.options['TimeLimit'] = TimeLimit


results = opt.solve(model, tee=True)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 20.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2020.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\User\AppData\Local\Temp\tmpj5ghswr4.cplex.log' open.
CPLEX> New value for time limit in seconds: 600
CPLEX> Problem 'C:\Users\User\AppData\Local\Temp\tmpg_y8ojyy.pyomo.lp' read.
Read time = 0.06 sec. (3.10 ticks)
CPLEX> Problem name         : C:\Users\User\AppData\Local\Temp\tmpg_y8ojyy.pyomo.lp
Objective sense      : Maximize
Variables            :    7297  [Nneg: 1153,  Binary: 6144]
Objective nonzeros   :     240
Linear constraints   :   30203  [Less: 21664,  Greater: 288,  Equal: 8251]
  Nonzeros           :  108113
  RHS nonzeros       :   21457

Variables            : Min LB: 0.

In [2502]:
for k in model.setK:
    
    for v in model.setV:

        for i in model.setN:

            for j in model.setN:

                if (pyo.value(model.x[i,j,k,v]))>0.01:

                    print(f'x[{i}, {j}, {k}, {v}] == {pyo.value(model.x[i,j,k,v])}')

x[0, 3, 0, 0] == 1.0
x[3, 12, 0, 1] == 1.0
x[12, 3, 0, 2] == 1.0
x[3, 13, 0, 3] == 1.0
x[13, 3, 0, 4] == 1.0
x[3, 13, 0, 5] == 1.0
x[13, 3, 0, 6] == 1.0
x[3, 4, 0, 7] == 1.0
x[4, 3, 0, 8] == 1.0
x[3, 3, 0, 9] == 1.0
x[3, 6, 0, 10] == 1.0
x[6, 14, 0, 11] == 1.0
x[1, 3, 1, 0] == 0.9999999999999997
x[3, 12, 1, 1] == 0.9999999999999997
x[12, 2, 1, 2] == 1.0
x[2, 12, 1, 3] == 1.0
x[12, 2, 1, 4] == 1.0
x[2, 13, 1, 5] == 1.0
x[13, 3, 1, 6] == 1.0
x[3, 3, 1, 7] == 1.0
x[3, 4, 1, 8] == 1.0
x[4, 2, 1, 9] == 1.0
x[2, 6, 1, 10] == 1.0
x[6, 15, 1, 11] == 1.0


In [2467]:
for k in model.setK:
    
    for v in model.setV:

        for i in model.setN:

            for j in model.setN:

                if ((pyo.value(model.x[i,j,k,v]))>0.01):

                    print(f'z[{j}, {k}, {v}] == {pyo.value(model.z[j,k,v])}')

z[3, 0, 0] == 10.000000000000004
z[12, 0, 1] == 10.0
z[3, 0, 2] == 10.0
z[13, 0, 3] == 10.0
z[3, 0, 4] == 10.0
z[13, 0, 5] == 10.0
z[3, 0, 6] == 10.0
z[4, 0, 7] == 10.0
z[3, 0, 8] == 9.999999999999996
z[3, 0, 9] == 0.0
z[6, 0, 10] == 10.0
z[14, 0, 11] == 0.0
z[3, 1, 0] == 20.0
z[12, 1, 1] == 19.999999999999993
z[2, 1, 2] == 20.0
z[12, 1, 3] == 20.0
z[2, 1, 4] == 20.000000000000004
z[13, 1, 5] == 20.0
z[3, 1, 6] == 20.0
z[3, 1, 7] == 0.0
z[4, 1, 8] == 20.0
z[2, 1, 9] == 19.999999999999993
z[6, 1, 10] == 20.0
z[15, 1, 11] == 0.0


In [2462]:
for k in model.setK:
    
    for v in model.setV:

        for i in model.setN:

            for j in model.setN:

                if ((pyo.value(model.x[i,j,k,v]))>0.01):

                    print(f'y[{j}, {k}, {v}] == {pyo.value(model.y[j,k,v])}')

y[3, 0, 0] == 10.000000000000004
y[12, 0, 1] == 0.0
y[3, 0, 2] == 10.0
y[13, 0, 3] == 0.0
y[3, 0, 4] == 10.0
y[13, 0, 5] == 0.0
y[3, 0, 6] == 10.0
y[4, 0, 7] == 0.0
y[3, 0, 8] == 9.999999999999996
y[3, 0, 9] == 10.0
y[6, 0, 10] == 0.0
y[14, 0, 11] == 0.0
y[3, 1, 0] == 20.0
y[12, 1, 1] == 0.0
y[2, 1, 2] == 20.0
y[12, 1, 3] == 0.0
y[2, 1, 4] == 20.0
y[13, 1, 5] == 0.0
y[3, 1, 6] == 20.0
y[3, 1, 7] == 20.0
y[4, 1, 8] == 0.0
y[2, 1, 9] == 20.0
y[6, 1, 10] == 0.0
y[15, 1, 11] == 0.0


In [2463]:
for k in model.setK:

    for v in model.setV:

        for i in model.setN:
            
            for j in model.setN:
            
                if pyo.value(model.x[i,j,k,v]) > 0.01:

                    print(f'w[{j}, {k}, {v}] == {pyo.value(model.w[j,k,v])}')
                    

w[3, 0, 0] == 27.72000000000007
w[12, 0, 1] == 63.51000000000003
w[3, 0, 2] == 99.29999999999995
w[13, 0, 3] == 125.0
w[3, 0, 4] == 150.70000000000005
w[13, 0, 5] == 176.4000000000001
w[3, 0, 6] == 202.10000000000014
w[4, 0, 7] == 237.20000000000016
w[3, 0, 8] == 272.3000000000002
w[3, 0, 9] == 272.3000000000002
w[6, 0, 10] == 314.0500000000002
w[14, 0, 11] == 352.6300000000002
w[3, 1, 0] == 33.3299999999999
w[12, 1, 1] == 69.11999999999989
w[2, 1, 2] == 93.05999999999995
w[12, 1, 3] == 117.0
w[2, 1, 4] == 140.94000000000005
w[13, 1, 5] == 179.36
w[3, 1, 6] == 205.06000000000006
w[3, 1, 7] == 205.06000000000006
w[4, 1, 8] == 240.16000000000008
w[2, 1, 9] == 274.4000000000001
w[6, 1, 10] == 312.98000000000013
w[15, 1, 11] == 354.73000000000013


In [2464]:
# Getting routes on solution

routes = [[] for k in model.setK]

for k in model.setK:
    
    for v in model.setV:

        for i in model.setN:

            for j in model.setN:

                if (pyo.value(model.x[i,j,k,v]))>0.01:
                    
                    if len(routes[k]) == 0:
                    
                        routes[k].extend([i,j])
                        
                    else:
                        
                        routes[k].append(j)

print(routes)

[[0, 3, 12, 3, 13, 3, 13, 3, 4, 3, 3, 6, 14], [1, 3, 12, 2, 12, 2, 13, 3, 3, 4, 2, 6, 15]]


In [2465]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib, random

random.seed(123)
colors = dict(matplotlib.colors.cnames.items())

hex_colors = tuple(colors.values())


fig = px.scatter(df_instance,x="x", y="y", symbol='type', symbol_sequence=["square","star","cross", "star", "x"], color='type', text=df_instance.index, hover_data={"x":False, "y":False,  "d":True, "type":True, "index":df_instance.index})

fig.update_traces(textposition='top center')

fig.update_layout(font=dict(size=9))

# Create scatter trace of text labels

for route in routes:
    
    color = random.choice(hex_colors)
    
    for index_node in range(1, len(route)):
        
        fig.add_shape(type="line",
            x0=df_instance.iloc[route[index_node-1]].x, 
            y0=df_instance.iloc[route[index_node-1]].y,
            x1=df_instance.iloc[route[index_node]].x,
            y1=df_instance.iloc[route[index_node]].y,

            line=dict(color=color,width=3), 
            opacity = 0.2,
            visible = True
        )
        


fig.show()


In [2457]:
import numpy as np

np.mean([np.mean([el for el in c[i] if el > 0]) for i in range(len(c))])

23.403275641025637

In [2458]:
w_b[0]/np.mean([np.mean([el for el in c[i] if el > 0]) for i in range(len(c))])

17.091624528782

In [2459]:
np.mean([demanda/10 for demanda in abs(d) if demanda > 0])*2

11.833333333333334

In [2460]:
np.sum([demanda/10 for demanda in abs(d) if demanda > 100])

14.0